# PYBOR
PYBOR is a multi-curve interest rate framework and risk engine based on multivariate optimization techniques, written in Python.

**Copyright &copy; 2017 Ondrej Martinsky, All rights reserved**

[www.github.com/omartinsky/pybor](https://github.com/omartinsky/pybor)

In [1]:
%pylab
%matplotlib inline
%run jupyter_helpers
%run yc_framework
figure_width = 16

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


## Pricing Curve Map
Generate pricing curvemap using stochastic short-rate model $dr_t=a(b-r_t)dt + \sigma dW_t$ for curves and tenor/cross-currency basis spreads. This will be our starting point, the curves inside this curvemap will be used only to reprice market instruments.

In [2]:
eval_date = create_date('2018-09-25')

In [3]:
class CurveConstructor:
    @staticmethod
    def FromShortRateModel(curve_id, times, r0, speed, mean, sigma, interpolation):
        import random
        times = array(times)
        assert_type(r0, float)
        assert_type(speed, float)
        assert_type(mean, float)
        assert_type(sigma, float)
        assert_type(interpolation, InterpolationMode)
        r = r0
        rates = []
        dts = times[1:] - times[:-1]
        dts = dts / 365.
        for dt in dts:
            rates.append(r)
            dr = speed * (mean - r) * dt + sigma * random.gauss(0, 1) * dt ** .5
            r += dr
        rates = array(rates)
        dfs_fwd = exp(-rates * dts)
        dfs = cumprod(dfs_fwd)
        return Curve(curve_id, times[0], times[1:], dfs, interpolation)

In [4]:
def generate_pricing_curvemap(eval_date):
    random.seed(0)
    pricing_curvemap = CurveMap()
    t = linspace(eval_date+0, eval_date+365*40, 7)
    def createCurve(name, r0, speed, mean, sigma):
        return CurveConstructor.FromShortRateModel(name, t, r0, speed, mean, sigma, interpolation=InterpolationMode.CUBIC_LOGDF)
    
    def createCurveFromSpread(baseCurve, name, r0, speed, mean, sigma):
        out = createCurve(name, r0, speed, mean, sigma)
        out.add_another_curve(baseCurve)
        return out
    
    u3m = createCurve('USDLIBOR3M', 0.02, 0.03, 0.035, 5e-4)
    u6m = createCurveFromSpread(u3m, 'USDLIBOR6M', 0.01, 0.03, 0.011, 5e-4)
    u12m = createCurveFromSpread(u6m, 'USDLIBOR12M', 0.01, 0.03, 0.011, 5e-4)
    g3m = createCurveFromSpread(u3m, 'GBPLIBOR3M', 0.03, 0.03, 0.0, 5e-4)
    u1b = createCurve('USD-USDOIS', 0.01, 0.03, 0.011, 5e-4)
    g1b = createCurveFromSpread(u1b, 'GBP-GBPSONIA', 0.005, 0.03, 0.005, 5e-4)
    gu1b = createCurveFromSpread(u1b, 'GBP-USDOIS', 0.001, 0.03, 0.001, 5e-4)
    pricing_curvemap.add_curve(u3m)
    pricing_curvemap.add_curve(u6m)
    pricing_curvemap.add_curve(u12m)
    pricing_curvemap.add_curve(g3m)
    pricing_curvemap.add_curve(g1b)
    pricing_curvemap.add_curve(u1b)
    pricing_curvemap.add_curve(gu1b)
    return pricing_curvemap

In [5]:
pricing_curvemap = generate_pricing_curvemap(eval_date)
usd3m = pricing_curvemap.curves_['USDLIBOR3M']
type(usd3m)
usd3m.get_all_dofs()

array([0.87517332, 0.74470161, 0.63218756, 0.53236615, 0.44141804,
       0.36584206])

## Curve Builder
Create the curve builder. Definitions of curves and market instruments from which these curves are built are loaded from the excel spreadsheet

In [6]:
curve_builder = CurveBuilder('engine_usd_gbp.xlsx', eval_date)

### Instrument Repricing
Use the curve builder (specifically instrument definitions which it contains) to **reprice** instruments from previously created **pricing curve map**.

Instrument *prices* are returned in a structure called *price ladder*

In [8]:
price_ladder = curve_builder.reprice(pricing_curvemap)

BaseException: Unable to get discount factor for dates [43368..57978] from curve with dates range [43368..57968]

### Display price ladder for a specific curve

In [ ]:
# Display:
figsize(figure_width, 4)
price_ladder.sublist('USDLIBOR3M').dataframe()
type(price_ladder)


### Display instrument par-rates
Every instrument type has a specific relationship between the quoted price $P$ and the par-rate $r$. For instance:

For interest rate swaps, $P = 100 \times r$

For interest rate futures, $P = 10000 \times (1 - r)$

The relationship between interest rate curve in a **zero-rate** space and instrument **par-rates** is often a source of confusion for many people. The below is a graph which illustrates the difference between USDLIBOR3M pricing curve's zero rates vs. par-rates of instruments (e.g. deposits, futures, swaps), which are repriced using this curve.
As we can see, only the par-rates of money market (deposit) instruments correspond to the curve points plotted in a zero-rate space.

In [ ]:
figsize(figure_width, 6)
m, r = curve_builder.get_instrument_rates(price_ladder.sublist('USDLIBOR3M'))
m = [exceldate_to_pydate(int(i)) for i in m]
title('USDLIBOR3M instrument par-rates')
linestyle(' '), plot(m,r,marker='.', label='USDLIBOR3M instrument par-rates')
linestyle('-'), pricing_curvemap['USDLIBOR3M'].plot()
legend();

In [ ]:
#price ladder
map = defaultdict(set)
for row in curve_builder.df_curves.iterrows():
    curve, stage = row[0], row[1]['Solve Stage']
    map[stage].add(curve)
stages = [map[i] for i in list(sorted(map))]
stages
stages[1]

In [ ]:
curve_builder = CurveBuilder('engine_usd_gbp.xlsx', eval_date)
curvemap = curve_builder.create_initial_curvemap(0.02)
instrument_prices = curve_builder.parse_instrument_prices(price_ladder)

In [ ]:
curves_for_stage = stages[1]

instruments_for_stage = curve_builder.get_instruments_for_stage(curves_for_stage)
dofs = curvemap.get_all_dofs(curves_for_stage)

for i in instruments_for_stage:
    print(i)
    r_actual = i.calc_par_rate(curvemap)
    price = instrument_prices[i.name_]
    r_target = i.par_rate_from_price(price)
    print(r_actual,price,r_target)

arguments = (curve_builder, curvemap, instrument_prices, curves_for_stage, instruments_for_stage)
bounds = (zeros(len(dofs)), numpy.inf * ones(len(dofs)))
solution = scipy.optimize.least_squares(fun=calc_residuals, x0=dofs, args=arguments, bounds=bounds)
print(solution.x)
assert isinstance(solution, scipy.optimize.OptimizeResult)

if not solution.success:
    raise BaseException(solution.message)
curvemap.set_all_dofs(curves_for_stage, solution.x)

In [ ]:
curve = curvemap.curves_['USDLIBOR3M']
type(curve)

### Curve Building
Build a brand new collection of curves from the instrument prices. This will take few seconds to complete ... 

In [ ]:
build_output = curve_builder.build_curves(price_ladder)

Below is the comparison of curves which we have just built (**solid lines**) with pricing curves (**dotted lines**). These lines should be as close to each other as possible.

In [ ]:
# Display:
figsize(figure_width, 6)
title('Curvebuilder output')
linestyle('solid'), build_output.output_curvemap.plot(), legend()
linestyle('dotted'), pricing_curvemap.plot();

### Instrument/Pillar Jacobian Matrix
The optimizer is using gradient-descent method to minimize error between instrument par-rates calculated from the curves which are subject to this optimization and the input instrument par-rates. In order to do this, optimizer calculates derivative ${\delta (I-I') / \delta P}$, where $I$ is the actual instrument par-rate, $I'$ is the target instrument par-rate and $P$ is the pillar value from the curve (practically speaking, the discount factor).

Jacobian matrix which is a by-product of the curve building process can be then used for risk calculation purposes and it will be illustrated lated.

In [ ]:
jacobian_dPdI = inv(build_output.jacobian_dIdP)
# Display:
figsize(figure_width, 8)
title("Jacobian Matrix"), xlabel('Pillars'), ylabel('Instruments')
imshow(jacobian_dPdI), colorbar();

## Risk Calculator
Risk calculator is constructed from the **curve builder** (which contains curve definitions and market conventions) and **build output** (which contains curves and the jacobian matrix).

In [ ]:
risk_calculator = RiskCalculator(curve_builder, build_output)

Let's define a convenience function which will bump par-rate of a specific instrument by the given amount of basis points and visualise the effect on all curves.

In [ ]:
def visualise_bump(instrument_search_string, bumpsize):
    instruments, bumpsize = risk_calculator.find_instruments(instrument_search_string), bumpsize  
    curvemap_bump = risk_calculator.get_bumped_curvemap(instruments, bumpsize, BumpType.JACOBIAN_REBUILD)

    # Display:
    figsize(figure_width, 6)
    linestyle('solid'), build_output.output_curvemap.plot(), legend()
    linestyle('dashed'), curvemap_bump.plot()
    title("Effect of bumping instrument %s" % instrument_search_string)

### Bumping Market Instruments

Bumping **market instruments** (such as those which define USD LIBOR 3M neutral curve) will cause parallel shift of all other curves which are defined as a basis from this curve

In [ ]:
visualise_bump('USDLIBOR3M__Swap__20Y', 1e-4)

In [ ]:
visualise_bump('USDLIBOR3M.*', 15e-4)

## Bumping Basis Instruments

Bumping **basis instruments** (USD LIBOR 6M) will cause movement in a USD LIBOR 6M basis curve

In [ ]:
visualise_bump('USDLIBOR6M__BasisSwap__20Y', 1e-4)

In [ ]:
visualise_bump('USDLIBOR6M.*', 15e-4)